In [1]:
from gemma import params as params_lib
path_checkpoint = '/home/austinwang/gemma_ckpt/2b/'
parameters = params_lib.load_and_format_params(path_checkpoint)


In [2]:
import sys
sys.path.append('/home/austinwang/austin_big_vision')
import big_vision.utils as u
import io
import numpy as np
local_np_save_path = '/home/austinwang/gemma2b.npz'

2024-08-13 19:09:11.990120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 19:09:12.011815: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 19:09:12.018512: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 19:09:13.129662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
ckpt = {'params': parameters['transformer']}
names_and_vals, _ = u.tree_flatten_with_names(ckpt)
names_and_vals = [(n, v.astype(np.float32)) for n, v in names_and_vals] # convert bf16 pos_emb to fp32 to avoid void np.dtype error
io_buffer = io.BytesIO()
np.savez(io_buffer, **{k: v for k, v in names_and_vals})
# save to local
with open(local_np_save_path, "wb") as f: f.write(io_buffer.getvalue())

In [4]:
model_p = u.npload(local_np_save_path)
for key in model_p.keys():
    print(key, model_p[key].shape, model_p[key].dtype)

params/embedder/input_embedding (256128, 2048) float32
params/final_norm/scale (2048,) float32
params/layer_0/attn/attn_vec_einsum/w (8, 256, 2048) float32
params/layer_0/attn/kv_einsum/w (2, 1, 2048, 256) float32
params/layer_0/attn/q_einsum/w (8, 2048, 256) float32
params/layer_0/mlp/gating_einsum (2, 2048, 16384) float32
params/layer_0/mlp/linear (16384, 2048) float32
params/layer_0/pre_attention_norm/scale (2048,) float32
params/layer_0/pre_ffw_norm/scale (2048,) float32
params/layer_1/attn/attn_vec_einsum/w (8, 256, 2048) float32
params/layer_1/attn/kv_einsum/w (2, 1, 2048, 256) float32
params/layer_1/attn/q_einsum/w (8, 2048, 256) float32
params/layer_1/mlp/gating_einsum (2, 2048, 16384) float32
params/layer_1/mlp/linear (16384, 2048) float32
params/layer_1/pre_attention_norm/scale (2048,) float32
params/layer_1/pre_ffw_norm/scale (2048,) float32
params/layer_10/attn/attn_vec_einsum/w (8, 256, 2048) float32
params/layer_10/attn/kv_einsum/w (2, 1, 2048, 256) float32
params/layer_1